In [1]:
!rm -rf mnist_data
!mkdir mnist_data
%cd mnist_data

!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

!gunzip *.gz
%cd ..

/Users/akshatgupta/Desktop/IIITD/SEM_6/DL/DL-Assignments/Assignment1/mnist_data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k  100 9680k    0     0  2989k      0  0:00:03  0:00:03 --:--:-- 2990k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881  100 28881    0     0  57863      0 --:--:-- --:--:-- --:--:-- 57877
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k  100 1610k    0     0  1555k      0  0:00:01  0:00:01 --:--:-- 1555k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542  100  4542    0     0   6634      0 --:--

In [10]:
import torch

def tensor_transform(data):
    return torch.from_numpy(data).to(torch.float32)

In [11]:
import os, struct, torch
import numpy as np


class CustomDataset:
    def __init__(self,root,train=True,transform=None):
        """
        Args:
            root (str): path to the directory of the dataset
            train (bool): if True return training data, else return testing data
            transform: transform function to apply to data
        """
        self.root = root
        self.train = train
        self.transform = transform

        ## File names for the train and test sets 
        self.data_name = ('train-images-idx3-ubyte' if train else 't10k-images-idx3-ubyte')
        self.labels_name = ('train-labels-idx1-ubyte' if train else 't10k-labels-idx1-ubyte')

        self.data, self.labels = self.load_data()


    def load_data(self):
        ## File paths for the data and label sets 
        data_path = os.path.join(self.root, self.data_name)
        labels_path = os.path.join(self.root, self.labels_name)

        with open(data_path, 'rb') as f:
            _, _, rows, cols = struct.unpack('>IIII', f.read(16))
            data = np.frombuffer(f.read(), dtype=np.uint8)
            data = data.reshape(-1, rows, cols)
        with open(labels_path, 'rb') as f:
            _, _ = struct.unpack('>II', f.read(8))
            labels = np.frombuffer(f.read(), dtype=np.uint8)

        return data, labels


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index):
        img, target = self.data[index], int(self.labels[index])

        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [12]:
data = CustomDataset(
    root="./mnist_data/",
    train=True,
    transform=tensor_transform
)